<img width="20%" alt="EarthDaily Agro" src="https://earthdaily.com/hubfs/EDA_logo_horizontal_gradient-navy_1.svg" style="border-radius: 12px" />

# Scaffold a new analytics task

Use `tools/analytics_scaffolder.py` to generate a client + task + registry wiring from an OpenAPI spec or a YAML description.

## Prerequisites

- Install the notebooks environment (includes `PyYAML` for YAML mode).
- Place your `openapi.json` or YAML config in a reachable path (e.g., `tmp/openapi.json` or `tmp/config.yaml`).
- `--path` is the endpoint path in the API (e.g., `/launch`), `--method` is the HTTP verb.
- `--target-root` defaults to `../lib/analytics`; override if you scaffold elsewhere.
- Optional toggles: `--use-crop-cycle` (use global crop_cycle.csv for season durations) and `--signature-cache` (hash inputs/geometry to skip identical runs).
- YAML supports `query:` and `body:` blocks (or `inputs:` for body only).


In [1]:
# See available options
!python ../tools/analytics_scaffolder.py --help

usage: analytics_scaffolder.py [-h] --mode {openapi,yaml} --source SOURCE
                               --task-name TASK_NAME
                               [--binding-key BINDING_KEY]
                               [--target-root TARGET_ROOT] [--path API_PATH]
                               [--method API_METHOD]
                               [--operation-id OPERATION_ID]
                               [--use-crop-cycle] [--signature-cache]
                               [--force]

Scaffold a new analytics task (client + task + registry wiring).

options:
  -h, --help            show this help message and exit
  --mode {openapi,yaml}
                        Input mode.
  --source SOURCE       Path to openapi.json or YAML config.
  --task-name TASK_NAME
                        snake_case task name, e.g., crop_mask_v2 (without
                        _task suffix).
  --binding-key BINDING_KEY
                        Override binding key (default: task-name).
  --target-root TARGET_ROOT

## Quick navigation
- [Prerequisites](#Prerequisites)
- [OpenAPI mode (example)](#OpenAPI-mode-(example))
- [YAML mode — step 1: craft the config](#YAML-mode-—-step-1:-craft-the-config)
- [YAML mode — step 2: scaffold the task](#YAML-mode-—-step-2:-scaffold-the-task)
- [After scaffolding](#After-scaffolding)

## OpenAPI mode (example)

Pick a path/method (or operationId) from your OpenAPI spec. Adjust variables then run the command below.

In [ ]:
mode = "openapi"
source = "../openapi/inseason_potential_score.json"
task_name = "inseason_potential_score"
api_path = "/launch"
api_method = "post"
target_root = "../lib/analytics"
use_crop_cycle = True  # set True if you need season duration/start from crop_cycle.csv
signature_cache = True  # set True to add <binding>_signature hash and skip identical rows

cmd = f"python ../tools/analytics_scaffolder.py --mode {mode} --source {source} --task-name {task_name} --path {api_path} --method {api_method} --target-root {target_root}"
if use_crop_cycle:
    cmd += " --use-crop-cycle"
if signature_cache:
    cmd += " --signature-cache"
print(cmd)
!$cmd

python ../tools/analytics_scaffolder.py --mode openapi --source ../tmp/openapi_inseason_potential_score.json --task-name inseason_potential_score --path /launch --method post --target-root ../lib/analytics --use-crop-cycle --signature-cache
Generated client at ..\lib\clients\inseason_potential_score_client.py
Generated task at ..\lib\analytics\tasks\inseason_potential_score_task.py
Updated params at ..\lib\analytics\params.py
Updated registry at ..\lib\analytics\registry.py
Review TODOs in generated files before committing.


In [ ]:
mode = "openapi"
source = "../openapi/greenness_detection.json"
task_name = "greenness_detection"
api_path = "/launch"
api_method = "post"
target_root = "../lib/analytics"
use_crop_cycle = True  # set True if you need season duration/start from crop_cycle.csv
signature_cache = True  # set True to add <binding>_signature hash and skip identical rows

cmd = f"python ../tools/analytics_scaffolder.py --mode {mode} --source {source} --task-name {task_name} --path {api_path} --method {api_method} --target-root {target_root}"
if use_crop_cycle:
    cmd += " --use-crop-cycle"
if signature_cache:
    cmd += " --signature-cache"
print(cmd)
!$cmd

python ../tools/analytics_scaffolder.py --mode openapi --source ../tmp/openapi.json --task-name greenness_detection --path /launch --method post --target-root ../lib/analytics --use-crop-cycle --signature-cache
Generated client at ..\lib\clients\greenness_detection_client.py
Generated task at ..\lib\analytics\tasks\greenness_detection_task.py
Updated params at ..\lib\analytics\params.py
Updated registry at ..\lib\analytics\registry.py
Review TODOs in generated files before committing.


## YAML mode — step 1: craft the config

Create or edit a YAML file listing the endpoint, inputs (with optional types), and outputs.

In [ ]:
%%bash
cat > tmp/config.yaml <<"YAML"
base_url: https://api.example.com
endpoint: /v1/predict
method: post
description: My task scaffolded from YAML
query:
  api_key:
    field: api_key
    type: str
body:
  id:
    field: id
    type: str
  geometry:
    field: geometry
    type: wkt
outputs:
  score: score
  status: status
YAML


## YAML mode — step 2: scaffold the task

Run the generator against the YAML config.

In [ ]:
mode = "yaml"
yaml_source = "tmp/config.yaml"
task_name = "my_task"
target_root = "notebooks_v2/lib/analytics"
use_crop_cycle = False
signature_cache = False

cmd = f"python ../tools/analytics_scaffolder.py --mode {mode} --source {yaml_source} --task-name {task_name} --target-root {target_root}"
if use_crop_cycle:
    cmd += " --use-crop-cycle"
if signature_cache:
    cmd += " --signature-cache"
print(cmd)
!$cmd

## After scaffolding

- Review generated files and TODOs:
  - `lib/clients/<task>_client.py`
  - `lib/analytics/tasks/<task>_task.py`
  - `lib/analytics/params.py` (new params stub)
  - `lib/analytics/registry.py` (appended block) and imports in `__init__.py`
- Adjust payload construction, output mapping, and any business logic.
- If the task depends on outputs from another analytics (e.g., needs an emergence date), ensure that column exists in the input dataframe or run that task first.
- In notebook (`3-use_case_analytics/use_case_analytics.ipynb`), call your new task via `run_task` with the binding key you chose, and check outputs/columns.
